### ***TextRank  and KeyPhrase Text Summarization***

---

###  Modules ###

In [10]:
import numpy as np
import helper as h

import networkx as nx

from tensorflow.keras.preprocessing import text as kpt
from tensorflow.keras.preprocessing import sequence

### Example text ###

In [11]:
text = '''
Głównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz. Akcja toczy się w roku 1602 roku gdzie wraz z resztą braci bierze udział w polowaniu na czarownice. Jednak niespodziewane spotkanie swojej przyrodniej siostry Anny Held, osądzanej o wykonywanie czarów, powoduje że Laymann odrzuca regułę klasztoru. Podstępnie zakrada się do biblioteki gdzie studiuje zakazane księgi. Zostaje przyłapany przez swojego nauczyciela brata Jakoba i uwięziony w lochu.
Główny bohater spotyka tam starszego człowieka Lugaida Vandroiy, który przedstawia mu się jako druid (Reach Out For The Light). Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii. W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny. Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy. Vandroiy używa go by przenieść Gabriela do równoległego świata.
W tym czasie biskup Mainz Johann Adam von Bicken, brat Jakob oraz rządca Falk von Kronberg są w drodze do Rzymu gdzie zamierzają spotkać się z papieżem Clemensem VIII (Glory of Rome). Niosą ze sobą także księgę odkrytą przez Gabriela. Wedle starożytnego zapisu wynika że księga jest ostatnią siódmą częścią pieczęci, która w całości daje właścicielowi absolutną wiedzę gdy tylko dostanie się on do wieży wyznaczającej środek Avantasii.
Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside). Opowiadają mu o toczącej się wojnie przeciwko siłom zła oraz o planach papieża (Sign Of The Cross). Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy. Gabriel przybywa w momencie gdy Clemens VIII rozmawia z tajemniczym głosem dochodzącym z wieży. Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower). Zdarzenie to kończy pierwszy album.
Jednak Gabriel nie jest usatysfakcjonowany. Chce dowiedzieć się więcej o świecie Avantasii dlatego Elderane wysyła nowicjusza do drzewa poznania. Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice). Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach. Kielich jest więzieniem dla ogromnej ilości torturowanych dusz, artefakt strzeżony jest także przez siejącą postrach bestię. Mimo niepowodzeń delfickich wypraw, Gabriel i Regrin powracają na ziemię by zmierzyć się z bestią. Przyjaciele znajdują kielich i przewracają go co umożliwia ucieczkę duszom. Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec.
Gabriel wraca do Vandroiya, który czekał na niego. Druid spełnia obietnice, zakrada się do więzienia by uwolnić Annę. Jednak znajduje tam „przemienionego” brata Jakoba który także chce uwolnić Annę. Falk von Kronberg nakrywa ich i każe aresztować. Rozpoczęła się walka w której poległ Vandroiy raniony przez Kronberga, który później zostaje uśmiercony przez brata Jakoba. Anna ucieka by ponownie złączyć się z Gabrielem. Podążają wspólnie nieznaną drogą w przyszłość (Into The Unknown).
'''

Source: https://pl.wikipedia.org/wiki/Avantasia

### ***1. TextRank Text Summarization*** ###

### Split text into sentences ###

Make a list of raw sentences as well as preprocessed sentences

In [3]:
sentences, raw_sentences = h.prepare_text(text)

In [12]:
sentences

['głównym bohaterem gabriel laymann nowicjusz klasztoru dominikanów mainz',
 'akcja toczy wraz resztą braci bierze udział polowaniu czarownice',
 'niespodziewane spotkanie swojej przyrodniej siostry anny held osądzanej wykonywanie czarów powoduje laymann odrzuca regułę klasztoru',
 'podstępnie zakrada biblioteki studiuje zakazane księgi',
 'zostaje przyłapany swojego nauczyciela brata jakoba uwięziony lochu',
 'główny bohater spotyka starszego człowieka lugaida vandroiy przedstawia druid reach out for the light',
 'spotkany człowiek opowiada gabrielowi innym zagrożonym wymiarze – avantasii',
 'zamian pomoc uratowaniu avantasii druid obiecuje uratowanie anny',
 'razem udaje uciec lochów breaking away czym vandroiy zabiera gabriela kamieniołomu ukryty portal łączący oba światy',
 'vandroiy używa przenieść gabriela równoległego świata',
 'czasie biskup mainz johann adam von bicken brat jakob rządca falk von kronberg drodze rzymu zamierzają spotkać papieżem clemensem glory of rome',
 'nios

### Sentence representation ###

Use keras tokenizer to produce sentence representation  

In [4]:
def make_representationX(sentences, max_top_words = 100, max_sent_len = 20):
    tokenizer = kpt.Tokenizer(num_words=max_top_words)
    tokenizer.fit_on_texts(sentences)
    seq = tokenizer.texts_to_sequences(sentences)
    return sequence.pad_sequences(seq, maxlen = max_sent_len, padding='post', truncating='post')

### Sentence similarity ###

Make sentence sepresentation of all sentences `(sentence_repr)` of shape(n,m) where:

n - number of sentences

m - max sentence length (parameter from `make_representationX`)

In [5]:
# zeroes 2x2 matrix with length equal to number of sentences 
sim_matrix = np.zeros((len(sentences), len(sentences)))


sentences_repr = make_representationX(sentences)

for n, sent1 in enumerate(sentences):
    for m, sent2 in enumerate(sentences):
        if sent1 != sent2:
            a = sentences_repr[n]
            b = sentences_repr[m]

            sim_matrix[n][m] =h.cos_sim_dist(a, b)    

In [6]:
scores = nx.pagerank(nx.from_numpy_array(sim_matrix))
summary = sorted(((scores[i], s) for i,s in enumerate(raw_sentences)), reverse = True)

In [7]:
RATIO = 0.3

In [8]:
summarized_sents = [sent[1] for sent in summary[:int(len(summary) * RATIO)]]
summarized_sents

['Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy',
 'Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach',
 'Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec',
 'Falk von Kronberg nakrywa ich i każe aresztować',
 'Wedle starożytnego zapisu wynika że księga jest ostatnią siódmą częścią pieczęci, która w całości daje właścicielowi absolutną wiedzę gdy tylko dostanie się on do wieży wyznaczającej środek Avantasii',
 'W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny',
 'Podstępnie zakrada się do biblioteki gdzie studiuje zakazane księgi',
 'Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice)',
 'Akcja toczy się w roku 1602 roku gdzie wraz z resztą braci bierze udział w polowaniu na czarownice',
 '\nGłównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w

In [9]:
h.display_highlights(raw_sentences, summarized_sents)

### ***2. KeyPhrase Text Summarization*** ###